# Neutralize features can make feature 0 really important

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install datatable > /dev/null
import datatable as dt
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read data

In [ ]:
%%time
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()
# train_data.fillna(train_data.mean(), inplace=True)

In [ ]:
train_data = train_data.loc[train_data.weight > 0].reset_index(drop = True)
train_data = train_data.query('date > 85').reset_index(drop = True)


features = [c for c in train_data.columns if 'feature' in c]
# f_mean = train_data[features[1:]].mean()
train_data[features[1:]] = train_data[features[1:]].fillna(method='ffill').fillna(0)
train_data['action'] = (train_data['resp'] > 0).astype('int')

# Utility function

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
#     print(f"count_i: {count_i}, Pi: {Pi}, t: {t}, u: {u}") 
    return u

# Neutralize features

In [ ]:
def neutralize_series(series: pd.Series, by=pd.Series, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    
    exposures = np.hstack((exposures, np.array([np.mean(series)]*len(exposures)).reshape(-1, 1)))
    correction = proportion*(exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    correction_scores = scores - correction
    neutralized = pd.Series(correction_scores.ravel(), index=series.index)
    return neutralized

In [ ]:
from tqdm.auto import tqdm

for f in tqdm(features):
    neut = neutralize_series(train_data[f], train_data['resp'])
    
    original_corr = np.corrcoef(train_data[f].values, train_data['resp'].values)[0, 1]
    neut_corr = np.corrcoef(neut, train_data['resp'].values)[0, 1]
    f_corr = np.corrcoef(neut, train_data[f].values)[0, 1]
    
    print(f'Origin: {original_corr:.3f}, After neutral: {neut_corr:.3f}, Corr with feature: {f_corr:.3f}')
    
    train_data[f] = neut.values

In [ ]:
# only feature 0
X = train_data[['feature_0']].values.reshape(-1, 1)
y = train_data['action'].values

In [ ]:
fold = 5
gkf = GroupTimeSeriesSplit(n_splits=fold)
splits = gkf.split(y, groups=train_data['date'].values)

In [ ]:
rmse_score = []
for i, (train_idx, val_idx) in enumerate(splits):
    clf = DecisionTreeClassifier(max_depth=3)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    
    score = mse(y_val, y_pred, squared=False)
    rmse_score.append(score)
    
    max_u = utility_score_bincount(
        train_data.loc[val_idx].date.values, 
        train_data.loc[val_idx].weight.values,
        train_data.loc[val_idx].resp.values,
        train_data.loc[val_idx].action.values,
    )
    
    u = utility_score_bincount(
        train_data.loc[val_idx].date.values, 
        train_data.loc[val_idx].weight.values,
        train_data.loc[val_idx].resp.values,
        y_pred,
    )
    
    
#     clf.save_model(f'xgboost_fold_{i}.model')
    print(f'Fold {i}, loss: {score}, u: {u}, max_u: {max_u}')


In [ ]:
import matplotlib.pyplot as plt
plt.plot(rmse_score)